In [ ]:
  # Correct import (requires python-dotenv package)

try:
    import requests
    import pandas as pd
    import time
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    apify_secret = session.sql("SELECT prod.raw.apify_api_key()").collect()[0][0]
except:
    import requests
    import pandas as pd
    import time
    import os
    from dotenv import load_dotenv
    load_dotenv()  # Load .env file
    results = os.getenv('apify_secret')  # Get environment variable

In [ ]:
import requests
import json
from typing import Optional, Dict, Any, List
from datetime import datetime


class Apify::
    """
    A simple client for interacting with Apify API v2 GET endpoints.
    
    This class provides methods to retrieve various types of data from Apify,
    including actors, datasets, key-value stores, and run information.
    """
    
    def __init__(self):
        """
        Initialize the Apify client with your API token.
        
        Args:
            api_token: Your Apify API token (found in Apify Console > Settings > Integrations)
        """
        self.headers = {
            "Authorization": f"Bearer {self.apify_secret}",
            "Content-Type": "application/json"
        }
    
    def _make_request(self, endpoint: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """
        Helper method to make GET requests to Apify API.
        
        Args:
            endpoint: The API endpoint (without base URL)
            params: Optional query parameters
            
        Returns:
            JSON response as a dictionary
            
        Raises:
            requests.exceptions.RequestException: If the request fails
        """
        url = f"{self.base_url}{endpoint}"
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making request to {url}: {e}")
            if hasattr(e.response, 'text'):
                print(f"Response body: {e.response.text}")
            raise
    
    # USER INFORMATION METHODS
    
    def get_user_info(self) -> Dict[str, Any]:
        """
        Get information about the authenticated user.
        
        Returns:
            User information including username, email, plan, etc.
        """
        print("Fetching user information...")
        return self._make_request("/users/me")
    
    # ACTOR METHODS
    
    def list_actors(self, limit: int = 10, offset: int = 0) -> Dict[str, Any]:
        """
        List actors owned by the authenticated user.
        
        Args:
            limit: Maximum number of actors to return
            offset: Number of actors to skip
            
        Returns:
            Dictionary containing list of actors and pagination info
        """
        print(f"Fetching actors (limit={limit}, offset={offset})...")
        params = {"limit": limit, "offset": offset}
        return self._make_request("/acts", params=params)
    
    def get_actor(self, actor_id: str) -> Dict[str, Any]:
        """
        Get detailed information about a specific actor.
        
        Args:
            actor_id: The actor ID or name (username/actor-name format)
            
        Returns:
            Detailed actor information
        """
        print(f"Fetching actor details for: {actor_id}")
        return self._make_request(f"/acts/{actor_id}")
    
    def get_actor_versions(self, actor_id: str) -> Dict[str, Any]:
        """
        Get all versions of a specific actor.
        
        Args:
            actor_id: The actor ID or name
            
        Returns:
            List of actor versions
        """
        print(f"Fetching versions for actor: {actor_id}")
        return self._make_request(f"/acts/{actor_id}/versions")
    
    # DATASET METHODS
    
    def list_datasets(self, limit: int = 10, offset: int = 0) -> Dict[str, Any]:
        """
        List datasets owned by the authenticated user.
        
        Args:
            limit: Maximum number of datasets to return
            offset: Number of datasets to skip
            
        Returns:
            Dictionary containing list of datasets and pagination info
        """
        print(f"Fetching datasets (limit={limit}, offset={offset})...")
        params = {"limit": limit, "offset": offset}
        return self._make_request("/datasets", params=params)
    
    def get_dataset(self, dataset_id: str) -> Dict[str, Any]:
        """
        Get information about a specific dataset.
        
        Args:
            dataset_id: The dataset ID or name
            
        Returns:
            Dataset information
        """
        print(f"Fetching dataset info for: {dataset_id}")
        return self._make_request(f"/datasets/{dataset_id}")
    
    def get_dataset_items(self, dataset_id: str, limit: int = 100, offset: int = 0,
                          fields: Optional[List[str]] = None) -> Dict[str, Any]:
        """
        Get items from a dataset.
        
        Args:
            dataset_id: The dataset ID or name
            limit: Maximum number of items to return
            offset: Number of items to skip
            fields: List of fields to include in the response
            
        Returns:
            Dictionary containing dataset items
        """
        print(f"Fetching items from dataset: {dataset_id}")
        params = {"limit": limit, "offset": offset}
        if fields:
            params["fields"] = ",".join(fields)
        return self._make_request(f"/datasets/{dataset_id}/items", params=params)
    
    # KEY-VALUE STORE METHODS
    
    def list_key_value_stores(self, limit: int = 10, offset: int = 0) -> Dict[str, Any]:
        """
        List key-value stores owned by the authenticated user.
        
        Args:
            limit: Maximum number of stores to return
            offset: Number of stores to skip
            
        Returns:
            Dictionary containing list of key-value stores
        """
        print(f"Fetching key-value stores (limit={limit}, offset={offset})...")
        params = {"limit": limit, "offset": offset}
        return self._make_request("/key-value-stores", params=params)
    
    def get_key_value_store(self, store_id: str) -> Dict[str, Any]:
        """
        Get information about a specific key-value store.
        
        Args:
            store_id: The store ID or name
            
        Returns:
            Key-value store information
        """
        print(f"Fetching key-value store info for: {store_id}")
        return self._make_request(f"/key-value-stores/{store_id}")
    
    def get_record(self, store_id: str, key: str) -> Any:
        """
        Get a specific record from a key-value store.
        
        Args:
            store_id: The store ID or name
            key: The key of the record to retrieve
            
        Returns:
            The value associated with the key (could be JSON, text, etc.)
        """
        print(f"Fetching record '{key}' from store: {store_id}")
        # Note: This endpoint might return different content types
        endpoint = f"/key-value-stores/{store_id}/records/{key}"
        url = f"{self.base_url}{endpoint}"
        
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        
        # Check content type to determine how to parse response
        content_type = response.headers.get('content-type', '')
        if 'application/json' in content_type:
            return response.json()
        else:
            return response.text
    
    def list_keys(self, store_id: str, limit: int = 100) -> Dict[str, Any]:
        """
        List all keys in a key-value store.
        
        Args:
            store_id: The store ID or name
            limit: Maximum number of keys to return
            
        Returns:
            Dictionary containing list of keys
        """
        print(f"Listing keys in store: {store_id}")
        params = {"limit": limit}
        return self._make_request(f"/key-value-stores/{store_id}/keys", params=params)
    
    # RUN METHODS
    
    def list_actor_runs(self, limit: int = 10, offset: int = 0, 
                        status: Optional[str] = None) -> Dict[str, Any]:
        """
        List actor runs for the authenticated user.
        
        Args:
            limit: Maximum number of runs to return
            offset: Number of runs to skip
            status: Filter by run status (READY, RUNNING, SUCCEEDED, FAILED, etc.)
            
        Returns:
            Dictionary containing list of runs
        """
        print(f"Fetching actor runs (limit={limit}, offset={offset}, status={status})...")
        params = {"limit": limit, "offset": offset}
        if status:
            params["status"] = status
        return self._make_request("/actor-runs", params=params)
    
    def get_run(self, run_id: str) -> Dict[str, Any]:
        """
        Get detailed information about a specific run.
        
        Args:
            run_id: The run ID
            
        Returns:
            Detailed run information
        """
        print(f"Fetching run details for: {run_id}")
        return self._make_request(f"/actor-runs/{run_id}")
    
    def get_run_log(self, run_id: str) -> str:
        """
        Get the log from a specific run.
        
        Args:
            run_id: The run ID
            
        Returns:
            The run log as a string
        """
        print(f"Fetching log for run: {run_id}")
        endpoint = f"/actor-runs/{run_id}/log"
        url = f"{self.base_url}{endpoint}"
        
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        return response.text


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")